In [77]:
!pip install google-search-results
!pip install pandas

In [82]:
from serpapi import GoogleSearch
from urllib.parse import *
import pandas as pd

params = {
  "engine": "google_maps",
  "q": "Toyota Auto2000",
  "ll": "@2.105857259955246, 99.82850985180268,14z",
  "type": "search",
  "api_key": "fed644f70c69d0dc4812218d3aca5facd7e4c092c539b7663c52575d3209e76a"
}

search = GoogleSearch(params)

lIdx = 0
lSum = 10
local_results = []

while lIdx <= lSum:
  results =search.get_dict()
  for Result in results["local_results"]:
    lIdx += 1

    if lIdx <= lSum:
      if Result ["reviews"]<200:
        lIdx += 1
        continue
      local_results.append({'Nama': Result["title"],
                            'data_id': Result["data_id"],
                            'total_reviews': Result["reviews"]});
    if "next" in results.get("serpapi_pagination",{}):
      search.params_dict.update(dict(parse_qsl(urlsplit(results.get("serpapi_pagination").get("next")).query)))
    else:
      break

In [80]:
for LR in local_results:
  print(LR)

{'Nama': 'Toyota Auto2000 Rantauprapat', 'data_id': '0x302da1ac3eab9c87:0x979ac3ca48e16c17', 'total_reviews': 241}
{'Nama': 'Toyota Auto2000 Krakatau Medan', 'data_id': '0x30313218b0962b9f:0xca9a1dd3262081a8', 'total_reviews': 437}
{'Nama': 'Toyota Auto2000 Medan Gatot Subroto', 'data_id': '0x30312e382266ab0b:0x5d7f19b5766fd368', 'total_reviews': 1554}
{'Nama': 'Toyota Auto2000 Medan Binjai', 'data_id': '0x3031297318f78655:0x9442a1d44c7c1c27', 'total_reviews': 644}
{'Nama': 'Toyota Auto2000 Sisingamangaraja', 'data_id': '0x3031304f7df093f1:0x5fd182e5fd5513b', 'total_reviews': 968}
{'Nama': 'AUTO 2000 Marelan', 'data_id': '0x3036cd5d54715efb:0xd5d7796e14b91d76', 'total_reviews': 512}


In [ ]:
# 2. Fetch reviews for the specific location (up to 200)
data = {'Nama_Showroom': [], 'name': [], 'rating': [], 'review': []}
review_params = {
    "engine": "google_maps_reviews",
    "data_id": "0x302da1ac3eab9c87:0x979ac3ca48e16c17",  # Use the extracted data_id
    "api_key" : "use-your-gmaps-API"
}

review_search = GoogleSearch(review_params)
reviews_fetched = 0
max_reviews = 200

while reviews_fetched < max_reviews:
    review_results = review_search.get_dict()
    if review_results.get("reviews"):  # Check if reviews are present in the current page
        for review in review_results["reviews"]:
            if reviews_fetched < max_reviews:  # Check review limit before adding
                data['Nama_Showroom'].append(place_name)  # Use the correct place name
                data['name'].append(review.get("user", {}).get("name", "Unknown User")) # Handle potential missing data
                data['rating'].append(review.get("rating")) # Handle potential missing data
                data['review'].append(review.get("snippet", "No Review")) # Handle potential missing data
                reviews_fetched += 1
            else:
                break  # Exit inner loop when review limit is hit
    else:
      print("No reviews found for this page or place.")
      break # Exit outer loop if no reviews

    if "next" in review_results.get("serpapi_pagination", {}):
        review_search.params_dict.update(dict(parse_qsl(urlsplit(review_results.get("serpapi_pagination").get("next")).query)))
    else:
        print("No more review pages found.")
        break  # Exit outer loop when no more pages

dfScrap = pd.DataFrame(data)
print(dfScrap) # Print the DataFrame for verification

# 3. Save to Excel
output_filename = f"{search_query}_reviews.xlsx"
dfScrap.to_excel(output_filename, index=False)
print(f"Reviews saved to {output_filename}")

                    Nama_Showroom                   name  rating  \
0    Toyota Auto2000 Rantauprapat       Akbar Dalimunthe     5.0   
1    Toyota Auto2000 Rantauprapat        Habibi Nasution     5.0   
2    Toyota Auto2000 Rantauprapat  Aslim Harapan Ritonga     5.0   
3    Toyota Auto2000 Rantauprapat           Rakimen Raki     5.0   
4    Toyota Auto2000 Rantauprapat           Bayu Anugrah     5.0   
..                            ...                    ...     ...   
195  Toyota Auto2000 Rantauprapat          David Tarigan     5.0   
196  Toyota Auto2000 Rantauprapat             Daud Diven     5.0   
197  Toyota Auto2000 Rantauprapat          Sarlan Sirait     5.0   
198  Toyota Auto2000 Rantauprapat          Arkan Ritonga     5.0   
199  Toyota Auto2000 Rantauprapat         Marhot Harahap     5.0   

                                                review  
0                                    Service excellent  
1                                    SERVICE EXCELLENT  
2           

In [ ]:
# Contoh data hanya untuk Auto2000 Rantauprapat
data = {
    'Nama_Showroom': ['Auto2000 Rantauprapat'] * 250,
    'name': [f'User{i}' for i in range(250)],
    'rating': [4] * 250,
    'review': [f'Review content {i}' for i in range(250)]
}

# Membuat DataFrame dari data
dfScrap = pd.DataFrame(data)

# Filter data untuk 'Auto2000 Rantauprapat' dan ambil 200 data pertama
filtered_df = dfScrap[dfScrap['Nama_Showroom'] == 'Auto2000 Rantauprapat'].head(200)

# Simpan hasil filter ke file .xlsx
output_path = 'auto2000_rantauprapat_reviews.xlsx'
filtered_df.to_excel(output_path, index=False, engine='openpyxl')

print(f"Data berhasil disimpan di {output_path}")


Data berhasil disimpan di auto2000_rantauprapat_reviews.xlsx
